#### NOTES: 

This script gets the Dice's coefficient (overlap) between "network"(ROI)-masked activations between pairs of contrasts/conditions.  

This script is a direct copy from NES_fMRI (Shari Liu and Kirsten Lydic) and has not been edited for generalizability or fully commented for clarity. For the time being, please ask Kirsten Lydic if you have any issues using this script. 

In [13]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns

import os.path as op
import os
import glob
import shutil
import datetime
import math
import subprocess
from nilearn import plotting
from nilearn import image
from nilearn import masking
import nilearn
from datetime import datetime

In [14]:
datestring = datetime.now()
print(datestring)
timestampStr = datestring.strftime("%b%d_%Y")
print(timestampStr)

2022-01-06 13:59:20.510021
Jan06_2022


In [15]:
# Get tasks and cons from your BIDS/code/contrasts.tsv file
# Second column is either 'description' or 'desc' in constrasts.tsv
def get_cons_and_tasks(proj_dir, second_column='description'):
    df = pd.read_csv(proj_dir + "data/BIDS/code/contrasts.tsv", sep="\t")
    cons = [[] for i in range(len(pd.unique(df['task'])))] 

    tasks = []
    for i, row in df.iterrows():
        if row['task'] not in tasks:
            tasks.append(row['task'])
    
    for i, row in df.iterrows():
        cons[tasks.index(row['task'])].append("%s" % (row[second_column].lower()))
    return cons, tasks

In [16]:
proj_dir = '/om2/group/saxelab/NES_fMRI/'

tier_dir = op.join(proj_dir, 'TIER')
copes_dir = op.join(tier_dir,'original_data','copes')
varcopes_dir = op.join(tier_dir,'original_data','varcopes')
zstats_dir = op.join(tier_dir,'original_data','zstats')

path_to_split_networks = op.join(proj_dir, 'TIER/analysis_data/networks/')

cons, tasks = get_cons_and_tasks(proj_dir)

subject_lists = op.join(proj_dir, 'scripts/10_whole_brain_group/subject_lists') 
subject_file = op.join(subject_lists, 'subjects_task-VOE.csv')
participant_IDs = pd.read_csv(subject_file, header=None)
filesubs = pd.unique(participant_IDs[0]).tolist()
        

In [17]:
network_names = ['visual', 'somatomotor', 'dorsal_attention', 'ventral_attention', 'limbic', 'frontoparietal', 'default']

for network in network_names:
    network_fname = op.join(path_to_split_networks,'{}.nii.gz'.format(network))
    img = image.load_img(network_fname)
    a = img.get_fdata()
    nonzero = np.count_nonzero(a)
    if nonzero == 0:
        print('{} is empty!'.format(network))
    if not np.array_equal(a, a.astype(bool)):
        print('{} is not binary'.format(network))

In [18]:
# create function to write warnings/errors to file 
warningfile = 'warnings' + timestampStr + '.txt'
print(warningfile)
#writeissue(warningfile, message)
def writeissue(filename, message):
    f = open(filename, 'a')
    f.write(message + '\n\n\n') 
    f.close()

warningsJan06_2022.txt


In [19]:
# create some directories for outputs 
outputs_dir =  op.join(tier_dir, 'analysis_data/overlap_analysis')
threshold_mask_outputdir = op.join(outputs_dir, 'subject_thresholded_masks')
k_and_z_thresholded_images_outputdir =  op.join(outputs_dir, 'subject_thresholded_k_and_z')
pernetwork_thresholded_masks_outputdir =  op.join(outputs_dir, 'subject_thresholded_masks_perNetwork')
os.makedirs(outputs_dir, exist_ok = True)
os.makedirs(threshold_mask_outputdir, exist_ok = True)
os.makedirs(pernetwork_thresholded_masks_outputdir, exist_ok = True)
os.makedirs(k_and_z_thresholded_images_outputdir, exist_ok = True)


# create output file name for CSV 
fname_outputCSV = op.join(outputs_dir, 'overlap_analysis_' + timestampStr + '.csv')

# load tasks and contrasts (we don't use tasks, but we need cons)
cons, tasks = get_cons_and_tasks(proj_dir)


In [20]:
tasks = ['VOE', 'VOEfam']
# cons, split by task
cons = [
    ['alltest-unexp-gt-exp', 'phystest-unexp-gt-exp', 'psychtest-unexp-gt-exp'], 
    ['fam1-gt-allelse']
]


comparisons = [
    ['alltest-unexp-gt-exp', 'fam1-gt-allelse'],
    ['phystest-unexp-gt-exp', 'psychtest-unexp-gt-exp']
]


comparison_names = ['visual_novelty_VOEalltest_vs_VOEfam1', 'phys_vs_psych_test_VOE']


In [21]:
def return_task_of_condition(comparisonI, contrastI):
    for i in range(len(cons)):
        if comparisons[comparisonI][contrastI] in cons[i]:
            task = tasks[i]
            return(task)    

In [22]:
z_cutoffs = [2.33]

In [23]:
## run command for cluster 

run_cluster = 0

if run_cluster:
    for sub in filesubs:
        print('SUBJECT: ', sub)

        for tasklayer in range(len(cons)):
            task = tasks[tasklayer]
            cons_in_task = cons[tasklayer]

            for con in cons_in_task:
                z_image_base = '{}/{}_{}_fold_{}_exclude_none_con_*_{}_zstat.nii.gz'.format(zstats_dir, sub, task, '*', con)
                matches = glob.glob(z_image_base)
                z_image = matches[0]
                for zthresh in z_cutoffs:
                    thresholded_image = '{}/{}_{}_{}_z-thresholded-{}_zstat.nii.gz'.format(threshold_mask_outputdir, sub, task, con, str(zthresh)) 
                    clustersize_image = '{}/{}_{}_{}_cluster_sizes_z-{}.nii.gz'.format(threshold_mask_outputdir, sub, task, con, str(zthresh))
                    cmd = ['cluster', '-z', z_image, '--zthresh={}'.format(str(zthresh)), '--othresh={}'.format(thresholded_image), '--osize={}'.format(clustersize_image)]

                    os.chdir(threshold_mask_outputdir)
                    subprocess.call(cmd)

                    # now run fslmaths to create a binary mask for k>=10 cluster size 
                    min_cluster_mask = '{}/{}_{}_{}_mincluster_mask_z-{}.nii.gz'.format(threshold_mask_outputdir, sub, task, con, str(zthresh))
                    cmd_fsl = ['fslmaths', '-dt', 'int', clustersize_image, '-thr', '10', '-bin', min_cluster_mask]
                    os.chdir(threshold_mask_outputdir)
                    subprocess.call(cmd_fsl)



In [24]:
def dices(comparison_data): 
    comp1 = comparison_data[0]
    comp1 = comp1.astype('float')
    # set 0s to np.nan so that they do not count towards "matches"
    comp1[comp1 == 0] = np.nan
    
    comp2 = comparison_data[1]
    comp2 = comp2.astype('float')
    comp2[comp2 == 0] = np.nan
    
    matches = comp1 == comp2
    matches = matches.astype(int)  
    counts_1 = sum(comparison_data[0])
    counts_2 = sum(comparison_data[1])
    counts_combined = counts_1 + counts_2
    overlap_size = sum(matches)
    dices_coef = 2*overlap_size/counts_combined 
    return dices_coef

In [28]:
for network in network_names: 
    
    network_fname = op.join(path_to_split_networks,'{}.nii.gz'.format(network))
    network_mask_img = image.load_img(network_fname)
    network_data = network_mask_img.get_fdata()
    raveled_network = np.ravel(network_data)
    network_n_noxels = sum(raveled_network)
    
    # network mask image is [0 or 1] where: only 1 if voxel in network
    
    print (network, network_n_noxels)
    

visual 21441.0
somatomotor 18797.0
dorsal_attention 14493.0
ventral_attention 13079.0
limbic 10718.0
frontoparietal 18788.0
default 28565.0


In [29]:
for network in network_names: 
    
    network_fname = op.join(path_to_split_networks,'{}.nii.gz'.format(network))
    network_mask_img = image.load_img(network_fname)
    network_data = network_mask_img.get_fdata()
    raveled_network = np.ravel(network_data)
    network_n_noxels = sum(raveled_network)
    
    # network mask image is [0 or 1] where: only 1 if voxel in network
    
    print (network, network_n_noxels)
    
    for sub in filesubs: 

        for zthresh in z_cutoffs:

            for comparisonI in range(len(comparisons)):
                comparison = comparisons[comparisonI]
                flat_images_to_compare = [[] for i in range(len(comparison))]
                nVoxels_in_masked_thresholded_zdata = [[] for i in range(len(comparison))]
                
                for contrastI in range(len(comparison)):
                    contrast = comparison[contrastI]

                    task = return_task_of_condition(comparisonI, contrastI)

                    path_to_thresholded_z = '{}/{}_{}_{}_z-thresholded-{}_zstat.nii.gz'.format(threshold_mask_outputdir, sub, task, contrast, str(zthresh)) 
                    thresholded_z_img = image.load_img(path_to_thresholded_z)
                    # z-thresholded image is [0 or <value>] where: only <value> if <value> >= zthresh

                    path_to_mincluster_mask = '{}/{}_{}_{}_mincluster_mask_z-{}.nii.gz'.format(threshold_mask_outputdir, sub, task, contrast, str(zthresh))
                    mincluster_image = image.load_img(path_to_mincluster_mask)
                    # mincluster image is [0 or 1] where: only 1 if <previous value, cluster size k> was >= 10 

                    # multiply voxel-thresholded by binary mincluster mask to match the 0 values
                    voxel_and_cluster_thresholded_zstat = image.math_img("img1 * img2", img1 = thresholded_z_img, img2 = mincluster_image)
                    
                    # save this to a file
                    voxel_and_cluster_thresholded_zstat_fname = '{}/{}_{}_{}_thresholded_cluster-10_z-{}_zstat.nii.gz'.format(k_and_z_thresholded_images_outputdir, sub, task, contrast, str(zthresh)) 
                    voxel_and_cluster_thresholded_zstat.to_filename(voxel_and_cluster_thresholded_zstat_fname)
                    
                    # now we have voxel and cluster thresholded image; 
                    # now multiply that by binary network mask to set any voxels not in network to 0 
                    network_masked_thresholdZ = image.math_img("img1 * img2", img1 = voxel_and_cluster_thresholded_zstat, img2 = network_mask_img)
                    
                    # save this to a file
                    network_masked_thresholdZ_fname = '{}/{}_{}_{}_cluster_thresholded_z-{}_network-{}_zstat.nii.gz'.format(pernetwork_thresholded_masks_outputdir, sub, task, contrast, str(zthresh), network) 
                    network_masked_thresholdZ.to_filename(network_masked_thresholdZ_fname)
                    
                    # extract an array from the resulting masked image 
                    network_masked_thresholdZ_data = network_masked_thresholdZ.get_fdata()
                    
                    # convert to a binary mask for comparisons 
                    contrast_boolean_mask = network_masked_thresholdZ_data > 0
                    contrast_binary_mask = contrast_boolean_mask.astype(int) # true/false to 1/0
                    
                    # flatten the 3d array (preserves relative indices across images)
                    raveled_mask_for_overlap = np.ravel(contrast_binary_mask)
                    
                    # get the number of voxels 
                    nVoxels_in_masked_thresholded_zdata[contrastI] = sum(raveled_mask_for_overlap)
                    
                    flat_images_to_compare[contrastI] = raveled_mask_for_overlap
                    
                # exit per-contrast loop
                # make comparison
                if (sum(flat_images_to_compare[0]) == 0) or (sum(flat_images_to_compare[1]) == 0): 
                    # at least one of the contrasts had no worthy voxels
                    dices_coef = np.nan 
                else: 
                    dices_coef = dices(flat_images_to_compare)
          
                df_mag_currentrow = pd.DataFrame({
                    'participantID': sub, 
                    'network': network, 
                    'network_n_noxels': network_n_noxels,
                    'z_threshold': zthresh,
                    'comparison': comparison_names[comparisonI], 
                    'contrast1': comparison[0],
                    'contrast2': comparison[1],
                    'con1_n_indices_thresholded_z': nVoxels_in_masked_thresholded_zdata[0],
                    'con2_n_indices_thresholded_z': nVoxels_in_masked_thresholded_zdata[1],
                    'overlap_dices_coef': dices_coef}, index=[0])
                
                if not os.path.isfile(fname_outputCSV):
                    # if file doesn't exist, write it with column headers 
                    df_mag_currentrow.to_csv(fname_outputCSV, index=False, header='column_names')
                else: 
                    # else append w/o column headers 
                    df_mag_currentrow.to_csv(fname_outputCSV, mode='a', index=False, header=False)





visual 21441.0
somatomotor 18797.0
dorsal_attention 14493.0
ventral_attention 13079.0
limbic 10718.0
frontoparietal 18788.0
default 28565.0


In [30]:
print("DONE!")

DONE!


In [31]:
df = pd.read_csv(fname_outputCSV)

In [32]:
df.head(50)

,participantID,network,network_n_noxels,z_threshold,comparison,contrast1,contrast2,con1_n_indices_thresholded_z,con2_n_indices_thresholded_z,overlap_dices_coef
0,sub-SAXNESs004,visual,21441.0,2.33,visual_novelty_VOEalltest_vs_VOEfam1,alltest-unexp-gt-exp,fam1-gt-allelse,948,7603,0.138229
1,sub-SAXNESs004,visual,21441.0,2.33,phys_vs_psych_test_VOE,phystest-unexp-gt-exp,psychtest-unexp-gt-exp,552,347,0.024472
2,sub-SAXNESs005,visual,21441.0,2.33,visual_novelty_VOEalltest_vs_VOEfam1,alltest-unexp-gt-exp,fam1-gt-allelse,1258,8680,0.165224
3,sub-SAXNESs005,visual,21441.0,2.33,phys_vs_psych_test_VOE,phystest-unexp-gt-exp,psychtest-unexp-gt-exp,324,1204,0.112565
4,sub-SAXNESs006,visual,21441.0,2.33,visual_novelty_VOEalltest_vs_VOEfam1,alltest-unexp-gt-exp,fam1-gt-allelse,124,10425,0.018390
5,sub-SAXNESs006,visual,21441.0,2.33,phys_vs_psych_test_VOE,phystest-unexp-gt-exp,psychtest-unexp-gt-exp,417,70,0.000000
6,sub-SAXNESs007,visual,21441.0,2.33,visual_novelty_VOEalltest_vs_VOEfam1,alltest-unexp-gt-exp,fam1-gt-allelse,410,4105,0.093023
7,sub-SAXNESs007,visual,21441.0,2.33,phys_vs_psych_test_VOE,phystest-unexp-gt-exp,psychtest-unexp-gt-exp,44,1676,0.000000
8,sub-SAXNESs008,visual,21441.0,2.33,visual_novelty_VOEalltest_vs_VOEfam1,alltest-unexp-gt-exp,fam1-gt-allelse,1491,10716,0.200705
9,sub-SAXNESs008,visual,21441.0,2.33,phys_vs_psych_test_VOE,phystest-unexp-gt-exp,psychtest-unexp-gt-exp,709,755,0.076503
